In [1]:
# pip install marker-pdf

In [2]:
import os
import re
import torch
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline, LlamaCpp
from langchain_core.documents import Document
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.config.parser import ConfigParser
import pypdfium2 as pdfium
from llama_cpp import Llama


os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

In [3]:
file_path = "SPRI_AI_Brief_2023년12월호_F.pdf"
num_chunk_size = 1000
num_chunk_overlap = 50

# step 1 : load document using marker-pdf
config_parser = ConfigParser({
    "output_format": "markdown",
    "paginate_output": True  # 페이지 구분 활성화
})
converter = PdfConverter(
    config=config_parser.generate_config_dict(),
    artifact_dict=create_model_dict(),
)

rendered = converter(file_path)

# 페이지별로 분리하여 LangChain Document로 변환
pages = re.split(r'\{(\d+)\}-+\n', rendered.markdown)
docs = []
for i in range(1, len(pages), 2):
    page_num = int(pages[i])
    content = pages[i + 1] if i + 1 < len(pages) else ""
    if content.strip():  # 빈 페이지 제외
        docs.append(Document(
            page_content=content.strip(),
            metadata={"source": file_path, "page": page_num}
        ))

# step 2 : split document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=num_chunk_size, chunk_overlap=num_chunk_overlap)
split_documents = text_splitter.split_documents(docs)

# step 3 : Embedding
# embeddings = OpenAIEmbeddings()
hf_embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3", model_kwargs={"device": "cuda"}, encode_kwargs={"normalize_embeddings": True},)

# step 4 : vector DB
try:
    vectorstore = FAISS.load_local(
        folder_path="faiss_db",
        index_name="faiss_index",
        embeddings=hf_embeddings,
        allow_dangerous_deserialization=True,
    )
except:
    vectorstore = FAISS.from_documents(documents=split_documents, embedding=hf_embeddings)
    vectorstore.save_local("faiss_db", "faiss_index")

# vectorstore.add_documents(new_split_documents)
# vectorstroe.save_local("faiss_db", "faiss_index")

/home/user/source/python312/.venv_python312/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]
Detecting bboxes: 0it [00:00, ?it/s]


In [23]:
# step 5 : Retriever Search
retriever = vectorstore.as_retriever()

# step 6 : generate prompt template
prompt_template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Question:
{question}

#Context:
{context}

#Answer:"""

# step 7 : LLM (llama_cpp 직접 사용)
llm = Llama.from_pretrained(
    repo_id="unsloth/gpt-oss-20b-GGUF",
    filename="gpt-oss-20b-Q4_K_M.gguf",
    n_gpu_layers=-1,
    n_ctx=16384,
    verbose=False,
)

# step 8: format_docs
def format_docs(docs):
    return "\n\n".join(
        f"[page {d.metadata.get('page', 0) + 1}] {d.page_content}" for d in docs
    )

# step 9: RAG 질의 함수 (스트리밍)
def ask(question: str):
    # retrieve
    docs = retriever.invoke(question)
    context = format_docs(docs)
    
    # prompt 생성
    prompt = prompt_template.format(question=question, context=context)
    
    # 스트리밍 출력 (생각 과정 포함)
    output = llm.create_chat_completion(
        messages=[{"role": "user", "content": prompt}],
        stream=True
    )
    
    full_response = ""
    for chunk in output:
        delta = chunk["choices"][0]["delta"]
        if "content" in delta:
            print(delta["content"], end="", flush=True)
            full_response += delta["content"]
    print()  # 줄바꿈
    
    # 답변 부분만 추출해서 다시 출력
    if '<|channel|>final<|message|>' in full_response:
        split_pos = full_response.find('<|channel|>final<|message|>')
        answer_only = full_response[split_pos + len('<|channel|>final<|message|>'):].strip()
    elif 'assistantfinal' in full_response:
        split_pos = full_response.find('assistantfinal')
        answer_only = full_response[split_pos + len('assistantfinal'):].strip()
    else:
        answer_only = full_response.strip()
    
    answer_only = answer_only.replace('<|end|>', '').strip()
    
    # 최종 답변
    print("\n=== 최종 답변 ===")
    print(answer_only)
    
    # 출처 페이지 출력 (유사도 순서, 중복 제거)
    seen = set()
    pages = []
    for d in docs:
        p = d.metadata.get('page', 0) + 1
        if p not in seen:
            seen.add(p)
            pages.append(p)
    print(f"\n📄 출처: {', '.join(f'page {p}' for p in pages)}")

llama_context: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified_iswa: using full-size SWA cache (ref: https://github.com/ggml-org/llama.cpp/pull/13194#issuecomment-2868343055)


In [24]:
ask("삼성이 만든 생성AI 의 이름은 무엇인가요?")

<|channel|>analysis<|message|>The question: "삼성이 만든 생성AI 의 이름은 무엇인가요?" Means: "What is the name of the generative AI created by Samsung?" The context indicates that Samsung's own generative AI is called "삼성 가우스" (Samsung Gauss). So answer: "삼성 가우스" in Korean. Provide answer in Korean.<|end|><|start|>assistant<|channel|>final<|message|>삼성에서 만든 생성 AI의 이름은 **“삼성 가우스”**입니다.

=== 최종 답변 ===
삼성에서 만든 생성 AI의 이름은 **“삼성 가우스”**입니다.

📄 출처: page 13, page 2


In [25]:
ask("AI 행동 강령에 대해 알려줘")

<|channel|>analysis<|message|>We need to answer: "AI 행동 강령에 대해 알려줘" (Tell me about the AI code of conduct). Use the provided context. Summarize key points. Provide in Korean. Should mention G7, Hiroshima AI Process, International Code of Conduct for Advanced AI Systems, key provisions: risk identification and mitigation across lifecycle, transparency, accountability, information sharing, collaboration, security controls, content verification, source confirmation. Also mention US executive order? But question is about AI 행동 강령. So focus on that. Provide explanation.<|end|><|start|>assistant<|channel|>final<|message|>**AI 행동 강령(International Code of Conduct for Advanced AI Systems)**  
- **배경**  
  - 2023년 10월 30일, G7(미국·일본·독일·영국·프랑스·이탈리아·캐나다) 정상회담에서 “히로시마 AI 프로세스”를 통해 마련된 국제적 규범입니다.  
  - 주요 목표는 **첨단 AI(기반모델·생성 AI 포함) 개발·배포 시 발생할 수 있는 위험을 사전·사후에 걸쳐 체계적으로 관리**하는 것입니다.

- **핵심 내용**  
  1. **위험 식별·완화**  
     - AI 수명주기 전반에 걸쳐 위험을 평가하고, 위험이 발생했을 때는 즉시 완화 조치를 취하도록 요구합니다.  
  2. **투명성·책임성**  